# <font color='#eb3483'> Intro to Deep Learning </font>
In this notebook we'll explore the basics of deep learning in Keras. Specifically, building a simple feed forward neural network and using it make predictions for a classification problem. General warning - neural networks are a massive topic (we could have done a whole course on them!), so this notebook will brush over some details. If you're interested in deep learning make sure to check out the resources linked in the deck to get a better understanding and start your journey as a deep learning practitioner.

In [ ]:
#The Usual Suspects
import numpy as np 
import pandas as pd 
import seaborn as sns
sns.set()

seed = 42
np.random.seed(seed)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <font color='#eb3483'> Pima Indians Data </font>
Before we jump in and start building a neural network, let's take a peak at the problem we're trying to solve - the [Pima Indians Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database) (diabetes.csv). Our goal is going to be diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset.

In [ ]:
# load Pima dataset
pima = pd.read_csv('data/diabetes.csv')
pima.head()

Looks like we have a pretty clean, well engineering data set. We have 8 features (everything but outcome) related to measurements about each patient. Let's check out how much data we have and do a quick view of the state of our data.

In [ ]:
#Number of samples
pima.shape

In [ ]:
# Any missing values?
pima.isna().sum()

In [ ]:
#Quick view of our numerical data
pima.describe()

In [ ]:
#Check out the class breakdown of our problem


Let's break our data intro one train/test split. 

In [ ]:
from sklearn.model_selection import train_test_split

features = list(pima.columns.values)
features.remove('Outcome')

X = pima[features]
y = pima['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print(X_train.shape)
print(X_test.shape)

This, especially by deep learning standards, isn't a ton of data but it's what we got!

## <font color='#eb3483'> Baseline Model </font>
One great thing to do (for any machine learning project) is to develop a good baseline for performance (i.e. how will you know if your model's actually improving?). It might be tempting to always turn to neural networks (they're pretty fricken cool and powerful) - but they aren't always needed. Let's start by just building a classifier using scikit-learn and see how it stacks up to our neural network.

In [ ]:
# Build a classifier here and evaluate it's accuracy!

## <font color='#eb3483'> Neural Network </font>
Alrighty, with our benchmark in place - let's go about building our first neural network! To build a network we're going to use Keras - a great high-level (i.e. doesn't make us worry too much about the details) deep learning framework.

### Step 1: Create a Model
The first step to creating a neural network in keras is to instantiate a model. Think of a model as your canvas that you can add layers too. There are a few different options for [model](https://keras.io/api/models/) in keras. We're just going to be using the Sequential model (let's us build simple models - just a list of layers).

In [ ]:
#Models are house in the keras.models module. We need to import Sequential
from keras.models import Sequential 

#Instantiates our model
#I.e. Creates a canvas we can add layers to
model = Sequential()

### Step 2: Add Layers
The next step is to add layers to our model. There are an exceedingly overwhelming number of potential [layer](https://keras.io/api/layers/) options in keras (including fancy things like convolutional layers, recurrent layers, regularizers...etc.). But we're going to start with the basics - a simple Dense layer (which is exactly what we covered in class, just a layer of normal "neurons).



![title](media/deep_network.png)

In [ ]:
#Layers are house in the keras.layers module. We need to import Dense
from keras.layers import Dense #Fully connected layer

In [ ]:
#Let's take a peak at our options
Dense?

We're going to specify the number of units (i.e. 'neurons') our layer has, and the activation function it uses. Remember that an activation function is just the function that takes are inputs x weights and turns that into an output. There are a bunch of different options (some functions you've probably never heard of)! We'll use relu for our 'hidden' layers (i.e. layers that aren't the output)
![title](media/activation_functions.png)

In [ ]:
# 1st layer: input_dim=8, 12 nodes, RELU
model.add(Dense(12, input_dim=8,  kernel_initializer = 'uniform', activation='relu'))
#Note that because this is the first layer we'll need to explicitly say the input dimension 
#(i.e. how many features we have)

Let's add another layer with less nodes.

In [ ]:
# 2nd layer: 8 nodes, RELU
model.add(Dense(8, kernel_initializer = 'uniform', activation='relu'))

And we want to end with an output layer (i.e. something that'll make our prediction). We want our output to be be between 0 and 1 (just like logistic regression) - so we'll use a sigmoid activation.

In [ ]:
# output layer: dim=1, activation sigmoid
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid' ))

And just like that - we've defined our neural network model! Woo!

**Your turn! I want everyone to try building a different model and we'll see who get's the best results at the end**

### Step 3: Compile Model
The next stage is to compile our model. Woah woah woah - what is compiling a model? Remember that Keras is a high-level deep learning framework - it doesn't make us worry about all the nitty gritty (i.e. how to represent our model for training, how to use computational resources like a gpu ...etc.). Compiling essentially sends our neural network blueprint to the packages that do the nitty gritty (like tensorflow or theano) to turn our blueprint into something that can run.

Let's see options for what we can specify when we compile:

In [ ]:
model.compile?

So there are a few things we should probably fiddle with:
- **Optimizer**: This defines how the neural network is trained (i.e. how we tweak those weights). There are a bunch of [options](https://keras.io/api/optimizers/) included in keras, but we'll use ADAM. This is one of those details we're going to gloss over, but optimization is a super intersting topic (chat with Connor - he's doing a PhD in it!).
- **Loss**: The loss function is how we're going to see how good our model is when training (i.e. our goal during training is to MINIMIZE the loss). We're going to use [binary cross entropy](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a). Sounds fancy but the idea is simple - if our prediction probabilities are bad we get a high loss.
- **Metrics**: Metrics are just performance measures we're going to ask Keras to track for us during training/testing. Let's ask it to track accuracy for us!

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',   # Binary cross entropy loss
             optimizer='adam', #Adam optimizer
             metrics=['accuracy']) #Track accuracy during training


**Feel free to try a different optimizer!**

### Step 4: Fit the Model
Alright our model's good to go - let's fit it! When we fit a neural network we're setting the weights in our network to try to minimize our loss function (just like the model's we've already learned!). Let's check out some options we can feed in.

In [ ]:
model.fit?

Woah! Keras let's us feed in our test data and report how good it does on the test data as we go. We can also specify a few things that are important to training:
- **Epochs**: Number of times we use our training data to adjust weights (i.e. how many times we ask the model to make a prediction on a training data, and then we correct the weights). The reason we put in data multiple times is related to the optimization, so we're going to kind of gloss over this but just remember more epochs == longer to train and we fiddle to make the weights even better.

- **Batch Size**: This is another piece of optimization jargon. Ignore it for now :)

In [ ]:
NB_EPOCHS = 1000  # num of epochs to test for
BATCH_SIZE = 16

print('Starting training...')
# train the model, store the results for plotting
history = model.fit(X_train, #Train X
                    y_train, #Train Y
                    validation_data=(X_test, y_test), #Test Data
                    epochs=NB_EPOCHS, # number of epochs
                    batch_size=BATCH_SIZE, #Batch size
                    verbose = 0) #Whether or not we want a bunch of updates from keras as it trains. We don't.
print('Bam - training done!')

Neural networks take some time to train... Let's do a mindful minute!


![title](media/mindful_minute.png)

### Step 5: Evaluate the Model
The last step is to see how well our model does! We've saved a history of how our model did during training - let's start by plotting both the accuracy and loss during training. 

In [ ]:
#Model Accuracy
sns.lineplot(y = history.history['accuracy'], x = np.arange(len(history.history['accuracy'])), 
             label = 'Train')
sns.lineplot(y = history.history['val_accuracy'], x = np.arange(len(history.history['accuracy'])),
            label = 'Test')
sns.mpl.pyplot.title('Model Accuracy')
sns.mpl.pyplot.ylabel('accuracy')
sns.mpl.pyplot.xlabel('epoch')

In [ ]:
# Model Loss
sns.lineplot(y = history.history['loss'], x = np.arange(len(history.history['accuracy'])), 
             label = 'Train')
sns.lineplot(y = history.history['val_loss'], x = np.arange(len(history.history['accuracy'])),
            label = 'Test')
sns.mpl.pyplot.title('Model Loss')
sns.mpl.pyplot.ylabel('Loss')
sns.mpl.pyplot.xlabel('epoch')

We'll take our best results (i.e. before we start overfitting). In deep learning lingo this is called early stopping!

In [ ]:
# print final accuracy
accuracy_hist = history.history['val_accuracy']
print("acc: " + str(max(accuracy_hist)) + " in epoch: " + str(accuracy_hist.index(max(accuracy_hist)) + 1))

So we see several things:

- ???

A few things could be done to improve the results:

- Different model architecture (num of nodes, etc)
- Model regularization (like dropout)
- Adaptive learning rate

Some of those words are probably foreign - if you're interested drop them a google after class!

### Step 6: Make Predictions
We have an awesome model - next step is to make predictions! Just like scikit-learn, keras models have a predict function.

In [ ]:
predictions = model.predict(X_test)
predictions[:5]

You'll notice the outputs are probabilities (which makes sense with our sigmoid function). We can also get the prediction class (i.e. binary 0/1) values.

In [ ]:
predictions = model.predict_classes(X_test)
predictions[:5]

And that's it! You've now built your own neural network, trained it, and used it to make predictions. Congrats!